# Statistical analysis of eye-tracking data

This notebook will cover:

1. [Some terminology](#1-some-terminology)
2. [Descriptive statistics](#2-descriptive-statistics)
3. [Inferential statistics](#3-inferential-statistics)

We will use [pymovements](https://pymovements.readthedocs.io/) to load the data and [statsmodels](https://www.statsmodels.org/) for statistical inference.

## 1. Some terminology

First of all, what is the difference between descriptive and inferential statistics?

| &nbsp;    | **Descriptive statistics**                                                    | **Statistical inference**                                                     |
| --------- | ----------------------------------------------------------------------------- | ----------------------------------------------------------------------------- |
| Aim:      | Describing/summarizing the **sample** of observations at hand                 | Drawing conclusions about the underlying **population** of the sample at hand |
| Methods:  | Data aggregation, data visualization, ...                                     | Probability theory, hypothesis testing, model fitting, model comparison, ...  |
| Examples: | Calculating mean/median/standard deviation/..., scatter plots, box plots, ... | t-test, linear regression, p-values, confidence intervals, ...                |

> **NOTE:** In machine learning, the term _inference_ means making **predictions** using a trained model. In statistics, _inference_ refers to the process of inferring properties of the population that our sample is drawn from, which often involves **fitting/training** a model. So when you see *statistical inference*, think *model training*.

When we use inferential statistics, we want to find out something about **the mechanism that generated the observations**. For example, we want to study the mechanism that generates eye movements while reading. Because this mechanism is very complex, we simplify it into a **model** that makes some assumptions. For example, we could assume that the lexical frequency of the words in the text has a linear effect on reading times. In this case, we would use:
- linear regression as our **model**.
- lexical frequency as our **independent variable**, and
- reading time as our **dependent variable**.

By fitting a linear regression, we will infer some properties of the underlying mechanism:
- The **slope** parameter represents the estimated strength of the effect (i.e., how quickly reading time changes when frequency increases)
- The **intercept** parameter represents the estimated value of the independent variable when the dependent variable is 0 (i.e., how quickly a word is read when the frequency is 0)
- The **error variance** parameter represents the estimates

Importantly, these are only **estimates** about the underlying population. We cannot know the true values for these properties exactly, since we only have access to a sample, not the entire population. This means that we will always have some **uncertainty** about the true values (the larger our sample is, the lower the uncertainty). We can describe or visualize the degree of uncertainty using **confidence intervals** (larger intervals = more uncertainty about the true values). It is also this uncertainty that allows us to **test hypotheses**. The **p-value** tells us how likely it is that the effect we observed was due to chance rather than due to the underlying mechanism. For example, if we want to test the significance of the slope in our linear regression model, we would calculate its p-value. If the p-value is 0.12, this means that there is a 12% probability that our estimated value for the slope is just because we got lucky when drawing our sample. The **significance level** is an arbitrarily chosen threshold to decide what we consider significant. If we choose a significance level of 0.05, and our p-value is below this number, we would say that the effect we found is *significant at the 0.05 level*.

Lastly, we can also compare different models to each other (**model comparison**). For example, if we wanted to test whether the effect between lexical frequency and reading time is linear or log-linear, we could fit one model where we transform the frequencies in to log-frequencies, and one model where we don't. We can then compare the **likelihood** of the two models to decide which is the more appropriate to use. The likelihood is the probability with which the model predicts the observed data. A higher likelihood indicates a better **model fit**.

## 2. Descriptive statistics

We're going to use a small eye-tracking dataset that already has precomputed reading measures: the University College London (UCL) corpus by [Frank et al. (2013)](https://doi.org/10.3758/s13428-012-0313-y). This dataset consists of 205 sentences read by 43 subjects. Each sentence is presented on a single line, one sentence per screen, followed by a comprehension check.

In [ ]:
import pymovements as pm

dataset = pm.Dataset("UCL", path="ucl-data").download().load()
reading_measures = dataset.precomputed_reading_measures[0].frame
reading_measures

First, let's plot a histogram of the reading measures to get an overview.

In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(2, 2, figsize=(12, 8))
axs[0, 0].hist(reading_measures["RTfirstfix"], bins=100)
axs[0, 0].set_xlabel("First-fixation time (ms)")
axs[1, 0].hist(reading_measures["RTfirstpass"], bins=100)
axs[1, 0].set_xlabel("First-pass time (ms)")
axs[0, 1].hist(reading_measures["RTrightbound"], bins=100)
axs[0, 1].set_xlabel("Right-bounded time (ms)")
axs[1, 1].hist(reading_measures["RTgopast"], bins=100)
axs[1, 1].set_xlabel("Go-past time (ms)")
plt.show()

Look at the figure above and try to answer these questions:

- Why are there so many zeros?
- Why does go-past time have much larger values than the other three measures?
- How are the measures distributed? (Are they normally distributed? Are the distributions unimodal?)
- Can you guess whether the mean or the median is larger for each measure? (Check your intuitions by calculating them.)

In [ ]:
# TODO: Calculate mean and median for each variable

Let's also look at the correlations between the reading measures:

In [ ]:
plt.scatter(reading_measures["RTfirstfix"], reading_measures["RTfirstpass"], alpha=0.5)
plt.xlabel("First-fixation time (ms)")
plt.ylabel("First-pass time (ms)")
plt.show()

- Can you explain why there is a perfectly diagonal line?
- Does this also happen for other pairs of reading measures?

Finally, let's calculate and analyze the lexical frequencies for each word:

In [ ]:
%pip install wordfreq

In [ ]:
import polars as pl
import wordfreq

reading_measures = reading_measures.with_columns(
    pl.col("word")
    .map_elements(
        lambda word: wordfreq.word_frequency(word, "en") * 100,
        return_dtype=pl.Float64,
    )
    .alias("word_freq")
)
reading_measures

In [ ]:
plt.hist(reading_measures["word_freq"], bins=50)
plt.xlabel("Word frequency (%)")
plt.show()

## 3. Inferential statistics

How does a word's frequency affect its reading time? We're going to analyze this question using two different approaches:

1. using a simple linear regression model with frequency as a fixed effect and reading time as the dependent variable, and
2. using a linear mixed-effects model that additionally includes the subject as a random effect.

### Simple linear regression

In [ ]:
%pip install statsmodels

The statsmodels package supports formulas similar to the ones used in [R](https://www.r-project.org/):

```R
y ~ x1 + x2 + x3
```

In this formula, `y` represents the dependent (outcome) variable, while `x1` and `x2` represent the independent (predictor) variables. The formula is translated into the following model:

$y = \beta_0 + \beta_1 x_1 + \beta_2 x_2 + \beta_3 x_3 + \epsilon$  

As you can see, an intercept $\beta_0$ is added by default. You can disable the intercept by using this formula:

```R
y ~ 0 + x1 + x2 + x3
```

Let's fit a model that predicts first-pass reading time from word frequency:

In [ ]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

model = smf.ols("RTfirstpass ~ word_freq", data=reading_measures)
results = model.fit()
results.summary()

`ols` means _ordinary least squares_, which refers to the method used to optimize the model parameters.

In the upper part of the summary, you can see the $R^2$ (proportion of explained variance) and the log-likelihood, which can be used as metrics for how well the model fits the data. $R^2$ is 0.043, meaning that this model can explain 4.3% of the variance in the first-pass reading times.

You can also see the two parameters (the `Intercept` and the slope for `word_freq`) listed in the lower part of the summary, with several interesting values:

- `coef`: the maximum likelihood estimate of the parameter/coefficient
- `std err`: the standard error, representing the uncertainty in the parameter value
- `t`: the t-statistic, which can be used to test the hypothesis that `coef != 0`
- `P>|t|`: the p-value for the t-test
- `[0.025 0.975]`: the 95% confidence interval; if 0 is outside this interval, the null hypothesis `coef == 0` can be rejected at the 0.05 level

Based on these values, we can conclude:
- The effect of word frequency are significantly larger than zero at $p < 0.001$.
- The effect of word frequency is negative, meaning that more frequent words take less time to read in the first pass.
- For every additional frequency percentage point, reading time increases on average by about 24 ms.

Let's plot the model's predictions to check how well they match the observed reading times:

In [ ]:
sm.graphics.plot_fit(results, "word_freq")
plt.show()

We can also plot the residuals (i.e., the error term for each data instance):

In [ ]:
plt.hist(results.resid, bins=50)
plt.xlabel("Error")
plt.show()

Our linear model assumes that the error terms are normally distributed, but clearly, this is not the case here. Apparently, our linear model does not fit the data very well. Here are some things you could try to improve it:

- Log-transform the independent or the dependent variable
- Removing data points where the word was skipped and only modeling the words that were actually fixated
- Averaging the reading measures over all subjects (to get rid of the between-subject variance)

> **NOTE:** The log-likelihood is only comparable between two models when they were fitted on the same data. Transforming the data is fine, but if you remove or aggregate data points, you will have to use other criteria for model comparison.

In [ ]:
# TODO: Modify the dataset, fit a new model, and visualize the results

### Linear mixed-effects model

Some readers might generally read faster than others, and some readers' reading speed might be more affected by word frequency than others'. To account for this variability between subjects, we can use a linear mixed-effects model.

For this model, we are grouping our dataset by subjects and adding a random intercept $\gamma_{0,subj}$ for each group:

$y = \beta_0 + \beta_1 wordfreq + \gamma_{0,subj} + \epsilon$

In [ ]:
model = smf.mixedlm(
    "RTfirstpass ~ word_freq", data=reading_measures, groups=reading_measures["subj_nr"]
)
results = model.fit()
results.summary()

In addition to the fixed intercept and slope as before, the summary also shows the variance for the random intercept (`Group Var`). This represents the amount of variability between subjects.

Note that there are no p-values or confidence intervals for the random intercepts, because random effects are typically only "nuisance parameters" to get rid of the variance that is not relevant for our hypothesis.

Let's visualize the regression lines for the first two subjects:

In [ ]:
import numpy as np

x = np.array([min(reading_measures["word_freq"]), max(reading_measures["word_freq"])])

for subj_nr, subj_effects in list(results.random_effects.items())[:2]:
    subj_intercept = subj_effects["Group"]
    color = plt.cm.tab10(subj_nr)
    subj_reading_measures = reading_measures.filter(pl.col("subj_nr") == subj_nr)
    plt.scatter(
        subj_reading_measures["word_freq"],
        subj_reading_measures["RTfirstpass"],
        marker=".",
        color=color,
        alpha=0.2,
    )
    # results.predict() only predicts based on the fixed effects,
    # so we have to add the random intercept ourselves
    y = results.predict({"word_freq": x}).to_numpy() + subj_intercept
    plt.plot(x, y, color=color)
plt.xlabel("Word frequency (%)")
plt.ylabel("First-pass time (ms)")
plt.show()

In [ ]:
# TODO: Apply the same modifications as for the simple linear regression model

To account for individual differences in the frequency effect, we can add a random slope $\gamma_{1,subj}$ to the model:

$y = \beta_0 + \beta_1 wordfreq + \gamma_{0,subj} + \gamma_{1,subj} wordfreq + \epsilon$

In [ ]:
model = smf.mixedlm(
    "RTfirstpass ~ word_freq",
    data=reading_measures,
    groups=reading_measures["subj_nr"],
    re_formula="~ word_freq",
)
results = model.fit()
results.summary()

The summary reports the variance of the random slope (`word_freq Var`), which represents how much the subjects vary in the way their reading times are affected by word frequencies. We also get the covariance between the random intercept and the random slope (`Group x word_freq Cov`).

Again, let's plot the regression lines for the first two subjects:

In [ ]:
for subj_nr, subj_effects in list(results.random_effects.items())[:2]:
    subj_intercept = subj_effects["Group"]
    subj_slope = subj_effects["word_freq"]
    color = plt.cm.tab10(subj_nr)
    subj_reading_measures = reading_measures.filter(pl.col("subj_nr") == subj_nr)
    plt.scatter(
        subj_reading_measures["word_freq"],
        subj_reading_measures["RTfirstpass"],
        marker=".",
        color=color,
        alpha=0.2,
    )
    # results.predict() only predicts based on the fixed effects,
    # so we have to add the random intercept and slope ourselves
    y = results.predict({"word_freq": x}).to_numpy() + subj_intercept + x * subj_slope
    plt.plot(x, y, color=color)
plt.xlabel("Word frequency (%)")
plt.ylabel("First-pass time (ms)")
plt.show()

If you want to exclude the random intercept and only include the random slope, use `re_formula="~ 0 + word_freq"`.

> **NOTE:** Unfortunately, statsmodels only supports one grouping for random effects. If you want to add random effects for, e.g., subjects *and* items, you have to use some [tricks](https://stackoverflow.com/a/59359776). Alternatively, you can install [R](https://www.r-project.org/) and use [Pymer4](https://eshinjolly.com/pymer4/) to interface with R packages that support more complex models.